In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

%matplotlib inline

In [12]:
training_data = [
    ("The cat ate the cheese".lower().split(), ["DET", "NN", "V", "DET", "NN"]),
    ("She read that book".lower().split(), ["NN", "V", "DET", "NN"]),
    ("The dog loves art".lower().split(), ["DET", "NN", "V", "NN"]),
    ("The elephant answers the phone".lower().split(), ["DET", "NN", "V", "DET", "NN"])
]

word2idx = {}

for sent, tags in training_data:
    
    for word in sent:
        
        if word not in word2idx:
            
            word2idx[word] = len(word2idx)

tag2idx = {"DET": 0, "NN": 1, "V": 2}

In [13]:
print(word2idx)

{'the': 0, 'cat': 1, 'ate': 2, 'cheese': 3, 'she': 4, 'read': 5, 'that': 6, 'book': 7, 'dog': 8, 'loves': 9, 'art': 10, 'elephant': 11, 'answers': 12, 'phone': 13}


In [14]:
import numpy as np

def prepare_sequence(seq, to_idx):
    
    idxs = [to_idx[w] for w in seq]
    idxs = np.array(idxs)
    
    return torch.from_numpy(idxs)

In [15]:
example_input = prepare_sequence("The dog answers the phone".lower().split(), word2idx)

print(example_input)

tensor([ 0,  8, 12,  0, 13])


In [16]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
         
        super(LSTMTagger, self).__init__()
        
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        
        self.hidden = self.init_hidden()

        
    def init_hidden(self):
         
        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))

    def forward(self, sentence):
         
        embeds = self.word_embeddings(sentence)
        
        
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        
        tag_outputs = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_outputs, dim=1)
        
        return tag_scores


In [17]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word2idx), len(tag2idx))

loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [18]:
test_sentence = "The cheese loves the elephant".lower().split()
 
inputs = prepare_sequence(test_sentence, word2idx)
inputs = inputs
tag_scores = model(inputs)
print(tag_scores)

_, predicted_tags = torch.max(tag_scores, 1)
print('\n')
print('Predicted tags: \n',predicted_tags)

tensor([[-0.8496, -1.1874, -1.3190],
        [-0.8723, -1.3023, -1.1708],
        [-0.8071, -1.4395, -1.1495],
        [-0.8483, -1.2315, -1.2729],
        [-0.8533, -1.2943, -1.2043]], grad_fn=<LogSoftmaxBackward0>)


Predicted tags: 
 tensor([0, 0, 0, 0, 0])


In [19]:
n_epochs = 300

for epoch in range(n_epochs):
    
    epoch_loss = 0.0
     
    for sentence, tags in training_data:
        
        model.zero_grad()

        model.hidden = model.init_hidden()

        sentence_in = prepare_sequence(sentence, word2idx)
        targets = prepare_sequence(tags, tag2idx)

        tag_scores = model(sentence_in)

        loss = loss_function(tag_scores, targets)
        epoch_loss += loss.item()
        loss.backward()
        
        optimizer.step()
        
    if(epoch%20 == 19):
        print("Epoch: %d, loss: %1.5f" % (epoch+1, epoch_loss/len(training_data)))

Epoch: 20, loss: 0.95663
Epoch: 40, loss: 0.72023
Epoch: 60, loss: 0.43272
Epoch: 80, loss: 0.24647
Epoch: 100, loss: 0.14244
Epoch: 120, loss: 0.09105
Epoch: 140, loss: 0.06393
Epoch: 160, loss: 0.04800
Epoch: 180, loss: 0.03782
Epoch: 200, loss: 0.03090
Epoch: 220, loss: 0.02593
Epoch: 240, loss: 0.02224
Epoch: 260, loss: 0.01940
Epoch: 280, loss: 0.01715
Epoch: 300, loss: 0.01534


In [20]:
test_sentence = "The cheese loves the elephant".lower().split()

inputs = prepare_sequence(test_sentence, word2idx)
inputs = inputs
tag_scores = model(inputs)
print(tag_scores)

_, predicted_tags = torch.max(tag_scores, 1)
print('\n')
print('Predicted tags: \n',predicted_tags)

tensor([[-1.2438e-02, -4.4293e+00, -7.7343e+00],
        [-5.0078e+00, -2.1539e-02, -4.2251e+00],
        [-4.6999e+00, -4.8742e+00, -1.6879e-02],
        [-4.6122e-03, -5.5998e+00, -7.0096e+00],
        [-6.4590e+00, -7.0574e-03, -5.2092e+00]],
       grad_fn=<LogSoftmaxBackward0>)


Predicted tags: 
 tensor([0, 1, 2, 0, 1])
